In [41]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pyautogui

Xlib.xauth: warning, no xauthority details available
Xlib.xauth: warning, no xauthority details available


In [24]:
def release_token(accessToken):
    server = 'api.nrgstream.com'
    path = '/api/ReleaseToken'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
    print('token released')

with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
print(default_pickle['accessToken'])
#release_token(default_pickle['accessToken'])

In [33]:
t = datetime(2022,1,1).date()
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (datetime.today().date(),monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (datetime.today().date(),daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:

with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle

{'accessToken': '',
 'current_data': (datetime.datetime(2022, 9, 7, 11, 36, 49, 310651),
                timeStamp  value      fuelType
  0   2022-09-07 00:00:00   1256          Coal
  1   2022-09-07 00:05:00   1260          Coal
  2   2022-09-07 00:10:00   1256          Coal
  3   2022-09-07 00:15:00   1257          Coal
  4   2022-09-07 00:20:00   1261          Coal
  ..                  ...    ...           ...
  135 2022-09-07 11:15:00    -59  Saskatchewan
  136 2022-09-07 11:20:00    -59  Saskatchewan
  137 2022-09-07 11:25:00    -59  Saskatchewan
  138 2022-09-07 11:30:00    -58  Saskatchewan
  139 2022-09-07 11:35:00    -58  Saskatchewan
  
  [1540 rows x 3 columns]),
 'daily_outage_dfs': [(datetime.date(2022, 9, 7),
        timeStamp  value  fuelType
   0   2022-09-02    290  Intertie
   1   2022-09-03      0  Intertie
   2   2022-09-04      0  Intertie
   3   2022-09-05      0  Intertie
   4   2022-09-06      0  Intertie
   ..         ...    ...       ...
   115 2022-12-26    

In [39]:
df = pd.read_csv('offsets_changes.csv').astype({'timeStamp':'datetime64[ns]','value':'int64','fuelType':'object'})
df.pivot(index='fuelType', columns='timeStamp', values='value')

timeStamp,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,...,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01
fuelType,,,,,,,,,,,,,,,,,,,,,
Biomass & Other,50,60,50,120,120,110,100,80,70,50,...,30,30,30,30,30,30,30,30,30,30
Coal,700,0,80,310,0,0,0,20,0,470,...,210,620,800,800,800,800,800,800,800,800
Dual Fuel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Energy Storage,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Hydro,120,140,120,140,150,70,30,50,80,80,...,40,0,0,0,0,0,0,0,0,0
Natural Gas,740,820,990,1760,1740,1530,1130,1150,1730,1250,...,470,450,80,30,90,1120,410,80,80,80
Solar,400,400,60,450,390,330,220,180,90,50,...,0,0,0,0,0,0,0,0,0,0
Wind,130,210,120,50,10,10,30,40,210,80,...,0,0,0,0,0,0,0,10,10,0


(4920, 1920)